In [1]:
import pandas as pd  #导入了Pandas库，并将其命名为pd

#使用Pandas中的read_excel函数读取了名为data.xlsx的Excel文件，并将其存储在名为df的Pandas DataFrame对象中
df = pd.read_excel('data/data.xlsx')

In [2]:
import jieba   #导入了jieba库

#使用Pandas中的apply函数，将df中的text列中的每一行数据，应用一个lambda函数。这个lambda函数使用了jieba库的cut函数，对每行数据进行分词，并将分词结果用空格连接成一个字符串。
#最后，将处理后的结果存储回df的text列中
df['text'] = df['text'].apply(lambda x: ' '.join(jieba.cut(x)))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ikkk\AppData\Local\Temp\jieba.cache
Loading model cost 0.576 seconds.
Prefix dict has been built successfully.


In [3]:
#打开了名为stopwords.txt的文件，使用utf-8编码，并将其存储在名为f的文件对象中。
#使用f.read()函数读取了f中的所有内容，然后使用splitlines()函数将其按行分割，并将结果存储在名为stopwords的列表中
with open('data/stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# 使用Pandas中的apply函数，将df中的text列中的每一行数据，应用一个lambda函数。这个lambda函数将每行数据按空格分割成一个列表，
# 然后使用列表推导式，将列表中不在stopwords中的单词筛选出来，并将其用空格连接成一个字符串
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))

In [4]:
#从sklearn.feature_extraction.text模块中导入了CountVectorizer类
from sklearn.feature_extraction.text import CountVectorizer

# 创建了一个名为vectorizer的CountVectorizer对象
vectorizer = CountVectorizer()
# 使用vectorizer.fit_transform()函数将df中的text列中的所有数据转换为词频矩阵X。
# 这个函数将文本数据转换为向量形式，每个单词在每个文本中出现的次数作为向量的一个维度，最终形成一个矩阵
X = vectorizer.fit_transform(df['text'])

In [5]:
# 从sklearn.naive_bayes模块中导入了MultinomialNB类
from sklearn.naive_bayes import MultinomialNB

# 定义了一个名为labels的列表，其中包含了六个字符串，分别表示六个分类标签
labels = ['味道好', '味道差', '态度好', '态度差', '卫生状态', '不新鲜']
# 定义了一个空字典classifiers，用于存储训练好的分类器
classifiers = {}
# 遍历labels列表中的每个标签
for label in labels:
    # df中获取当前标签列的数据，并将其存储在名为y的变量中
    y = df[label]
    # 创建了一个MultinomialNB对象clf
    clf = MultinomialNB()
    # 使用clf.fit()函数，将词频矩阵X和当前标签列的数据y作为参数，训练一个朴素贝叶斯分类器
    clf.fit(X, y)
    # 将训练好的分类器clf存储在classifiers字典中，以当前标签为键
    classifiers[label] = clf

In [6]:
# 定义了一个名为new_text的字符串变量，表示要进行分类的新文本
new_text = '这家店的味道很好，态度也很好，但是卫生有点问题'
# 使用jieba库的cut函数，对new_text进行分词，并将分词结果用空格连接成一个字符串。
new_text = ' '.join(jieba.cut(new_text))
# 使用列表推导式，将第二行得到的字符串按空格分割成一个列表，并筛选出不在stopwords中的单词，并将其用空格连接成一个字符串。这一行的作用是去除停用词
new_text = ' '.join([word for word in new_text.split() if word not in stopwords])

In [7]:
# 使用CountVectorizer对象vectorizer的transform函数，将处理后的新文本new_text转换为词频矩阵new_X。
# 这个函数将新文本转换为向量形式，每个单词在新文本中出现的次数作为向量的一个维度，最终形成一个矩阵。
# 注意，此处的new_text需要用列表包裹起来，表示要转换的是一个列表中的文本数据
new_X = vectorizer.transform([new_text])

In [8]:
# 开始一个for循环，遍历classifiers字典中的每个键值对，其中键表示分类标签，值表示训练好的朴素贝叶斯分类器
for label, clf in classifiers.items(): 
    # 使用当前分类器clf的predict_proba函数，对新文本的词频矩阵new_X进行预测，并得到该文本属于当前分类标签的概率proba。
    # 这个函数返回一个数组，第一维表示样本数，第二维表示类别数。
    # 由于此处只有一个样本，所以使用[0, 1]来取得该样本属于当前分类标签的概率。
    proba = clf.predict_proba(new_X)[0, 1] 
    # 使用print函数，输出当前分类标签和对应的概率。
    # 其中，f-string格式化字符串中的{label}和{proba:.2f}分别表示分类标签和概率，:.2f表示保留两位小数
    print(f'{label}的概率为{proba:.2f}') 

味道好的概率为0.99
味道差的概率为0.00
态度好的概率为0.20
态度差的概率为0.02
卫生状态的概率为0.96
不新鲜的概率为0.00


In [9]:
#测试 

In [21]:
text = '一般但是食材不新鲜，偶尔吃可以，不建议经常吃,鸡肉就算了，闻起来有点发臭，店内不太卫生' 
text = ' '.join(jieba.cut(text)) 
text = ' '.join([word for word in text.split() if word not in stopwords]) 
new_X = vectorizer.transform([text])  
for label, clf in classifiers.items(): 
    proba = clf.predict_proba(new_X)[0, 1] 
    print(f'{label}的概率为{proba:.2f}') 

味道好的概率为0.01
味道差的概率为0.47
态度好的概率为0.48
态度差的概率为0.00
卫生状态的概率为0.78
不新鲜的概率为0.70


In [ ]:
# 训练好的朴素贝叶斯分类器字典和CountVectorizer对象被存储到了两个pickle文件中，以便后续使用

In [23]:
# 导入了pickle库
import pickle
# 使用pickle库的dump函数，将训练好的朴素贝叶斯分类器字典classifiers存储到名为classifiers.pickle的文件中。
# 其中，'wb'表示以二进制写入模式打开文件
with open('classifiers.pickle', 'wb') as f:
    pickle.dump(classifiers, f)
# # 使用pickle库的dump函数，将CountVectorizer对象vectorizer存储到名为vectorizer.pickle的文件中。其中，'wb'表示以二进制写入模式打开文件
with open('data/vectorizer.pickle', 'wb') as f:
    pickle.dump(vectorizer, f)